<a href="https://colab.research.google.com/github/kazim-glitch/Deep-learning/blob/main/Simple_Autoencoders_In_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim

In [ ]:
import numpy

In [ ]:
from torchvision import datasets,transforms
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.ToTensor()

# transform = transforms.Compose([
#            transforms.ToTensor(),
#            transforms.Normalize((0.5),(0.5))                     
# ])

mnist_data = datasets.MNIST(root='./data',download=True,train=True,transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data,batch_size=64,shuffle=True)

In [ ]:
dataiter = iter(data_loader)

images,labels = dataiter.next()

print(torch.min(images),torch.max(images))

In [ ]:
class Autoencoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.Linear(28*28,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,12),
        nn.ReLU(),
        nn.Linear(12,3),
        
    )
    self.decoder = nn.Sequential(
        nn.Linear(3,12),
        nn.ReLU(),
        nn.Linear(12,64),
        nn.ReLU(),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128,28*28),
        nn.Sigmoid()
        
    )
  def forward(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    return decoded

    
      

In [ ]:
model = Autoencoder()

criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5)


In [ ]:
num_epoch =10
output = []

for epoch in range(num_epoch):
  for (img, _) in data_loader:
    img=img.reshape(-1,28*28)
    recon = model(img)
    loss = criterion(recon,img)

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  print(f'Epoch:{epoch+1},Loss:{loss.item():.4f}')
  output.append((epoch,img,recon))



In [ ]:
for k in range(0,num_epoch,4):
  plt.figure(figsize=(9,2))
  plt.gray()
  imgs=output[k][1].detach().numpy()
  recon=output[k][2].detach().numpy()
  for i,item in enumerate(imgs):
    if i>= 9: break
    plt.subplot(2,9,i+1)
    item=item.reshape(-1,28,28)
    plt.imshow(item[0])

  for i,item in enumerate(recon):
    if i >= 9:break
    plt.subplot(2,9,9+i+1)
    item=item.reshape(-1,28,28)
    plt.imshow(item[0])
